# 4) Analyze and evaluate optimization output

This final notebook uses the `runs.pkl` file created in notebook 2 and it analyzes:

- the distance between different feature sets in the parameter space
- the distance between different feature sets in the feature space
- the distance between different feature sets in the extracellular signals

In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import sys
import shutil
from scipy.spatial import distance

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import matplotlib.pyplot as plt
import MEAutility as mu
import json
import time
import numpy as np
import LFPy
from pathlib import Path

import model
import evaluator
import plotting 
import utils

%matplotlib notebook

### Load gt params and optimization output

In [ ]:
probe_type = "planar"
model_name = 'hay' # "hay"
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
data_folder = Path('..') / "data" / f"{model_name}_ecode_probe_{probe_type}"
result_folder = data_folder / "results" 

In [ ]:
for p in result_folder.iterdir():
    print(p.name)

In [ ]:
# compile model-specific mechanisms
if not (model_folder / "x86_64").is_dir():
    curr_dir = Path(".").absolute()
    os.chdir(model_folder)
    print(os.getcwd())
    os.system("nrnivmodl mechanisms")
    os.chdir(curr_dir)
else:
    print(f"Mechanisms already compiled for {model_name}")
    compiled_folder = Path("./x86_64")
    if compiled_folder.is_dir():
        shutil.rmtree(compiled_folder)
    shutil.copytree(model_folder / "x86_64", compiled_folder)

In [ ]:
cell = model.create(model_name=model_name, release=False)
cell_release = model.create(model_name=model_name, release=True)

probe = model.define_electrode(probe_type=probe_type)
# probe = None

param_names = [param.name for param in cell.params.values() if not param.frozen]
if model_name == "hallermann":
    cvode_active = False
else:
    cvode_active = True
sim = ephys.simulators.LFPySimulator(cell, cvode_active=cvode_active, electrode=probe)

params_release = {}
for param in cell_release.params_by_names(param_names):
    params_release[param.name] = param.value

In [ ]:
data = pickle.load(open(result_folder / 'runs.pkl', 'rb'))
df_optimization = pd.DataFrame(data)

In [ ]:
params_release

# Load protocols and original features

In [ ]:
protocols_file = data_folder / "efeatures" / "protocols_BPO.json"
feature_file = data_folder / "efeatures" / "features_BPO.json"

In [ ]:
extra_kwargs = dict(fs=20,
                    fcut=[300, 6000],
                    filt_type="filtfilt",
                    ms_cut=[3, 10])

In [ ]:
eva_soma = evaluator.create_evaluator(
    model_name=model_name,
    feature_set="soma",
    feature_file=str(feature_file)
)

In [ ]:
eva_extra = evaluator.create_evaluator(
    model_name=model_name,
    feature_set="extra",
    feature_file=str(feature_file),
    probe_type=probe_type,
    protocols_with_lfp="firepattern_200",
    **extra_kwargs
)

In [ ]:
opt_soma = df_optimization.query("feature_set == 'soma'")
opt_extra = df_optimization.query("feature_set == 'extra'")
print(f"Somatic optimizations: {len(opt_soma)}")
print(f"Extra optimizations: {len(opt_extra)}")

In [ ]:
fig, ax = plt.subplots()

for index, opt in opt_soma.iterrows:
    nevals = opt.nevals
    fitness = [lb["avg"] fo]
    ax.plot(opt.logbook[])

# Compute release responses

In [ ]:
responses_release = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), param_values=params_release)

In [ ]:
eap_release = utils.calculate_eap(responses=responses_release, protocols=eva_extra.fitness_protocols, 
                                  protocol_name="firepattern_200", **extra_kwargs)

In [ ]:
features_release = {}

In [ ]:
for obj in eva_extra.fitness_calculator.objectives:
    if len(obj.features) == 1:
        if "soma" in obj.features[0].name:
            feat_value = obj.features[0].calculate_feature(responses_release)
            features_release[obj.features[0].name] = feat_value
    else:
        print(f"More than one feature for objective: {obj.name}")

In [ ]:
features_release

# Example somatic optimization

In [ ]:
params_sample_soma = opt_soma.iloc[0]
params_sample_soma_dict = {k: v for k, v in zip(param_names, params_sample_soma.best_params)}
display(params_sample_soma)

In [ ]:
params_sample_soma.logbook[0].keys()

In [ ]:
responses_soma = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), param_values=params_sample_soma_dict)

In [ ]:
eap_soma = utils.calculate_eap(responses=responses_soma, protocols=eva_extra.fitness_protocols, 
                               protocol_name="firepattern_200", **extra_kwargs)

In [ ]:
plotting.plot_multiple_responses([responses_release, responses_soma], colors=["k", "C1"])

In [ ]:
plotting.plot_multiple_eaps([responses_release, responses_soma], eva_extra.fitness_protocols, probe,
                            protocol_name="firepattern_200", colors=["k", "C1"])

In [ ]:
eap_dist_soma = distance.cosine(eap_release.ravel(), eap_soma.ravel())

## Extra example

In [ ]:
params_sample_extra = opt_extra.iloc[1]
params_sample_extra_dict = {k: v for k, v in zip(param_names, params_sample_extra.best_params)}
display(params_sample_extra)

In [ ]:
responses_extra = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), param_values=params_sample_extra_dict)

In [ ]:
eap_extra = utils.calculate_eap(responses=responses_extra, protocols=eva_extra.fitness_protocols, 
                                protocol_name="firepattern_200", **extra_kwargs)

In [ ]:
plotting.plot_multiple_responses([responses_release, responses_extra], colors=["k", "C2"])

In [ ]:
plotting.plot_multiple_eaps([responses_release, responses_extra], eva_extra.fitness_protocols, probe,
                            protocol_name="firepattern_200", colors=["k", "C2"])

In [ ]:
eap_dist_extra = distance.cosine(eap_release.ravel(), eap_extra.ravel())

In [ ]:
print(eap_dist_soma, eap_dist_extra)

## Run protocols and compute features for all tested models

In [ ]:
feats_soma = []
eaps_soma = []
params_soma = []
responses_soma = []
for i, (index, opt) in enumerate(opt_soma.iterrows()):
    print(f"Computing {i + 1} / {len(opt_soma)}")
    # get best params
    params_dict = {k: v for k, v in zip(param_names, opt.best_params)}
    params_soma.append(params_dict)
    
    # run protocols
    responses = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), 
                                        param_values=params_dict) 
    responses_soma.append(responses)
    try:
        eap = utils.calculate_eap(responses=responses, protocols=eva_extra.fitness_protocols, 
                                  protocol_name="firepattern_200", **extra_kwargs)
    except:
        eap = np.zeros(eaps_soma[-1].shape)
    feat_dict = {}
    for obj in eva_extra.fitness_calculator.objectives:
        if len(obj.features) == 1:
            if "soma" in obj.features[0].name:
                feat_value = obj.features[0].calculate_feature(responses)
                feat_dict[obj.features[0].name] = feat_value
        else:
            print(f"More than one feature for objective: {obj.name}")
            
    feats_soma.append(feat_dict)
    eaps_soma.append(eap)

In [ ]:
feats_extra = []
eaps_extra = []
params_extra = []
responses_eap = []

for i, (index, opt) in enumerate(opt_extra.iterrows()):
    print(f"Computing {i + 1} / {len(opt_soma)}")
    # get best params
    params_dict = {k: v for k, v in zip(param_names, opt.best_params)}
    params_extra.append(params_dict)
    
    # run protocols
    responses = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), 
                                        param_values=params_dict)  
    responses_eap.append(responses)
    try:
        eap = utils.calculate_eap(responses=responses, protocols=eva_extra.fitness_protocols, 
                                  protocol_name="firepattern_200", **extra_kwargs)
    except:
        eap = np.zeros(eaps_extra[-1].shape)
        
    feat_dict = {}
    for obj in eva_extra.fitness_calculator.objectives:
        if len(obj.features) == 1:
            if "soma" in obj.features[0].name:
                feat_value = obj.features[0].calculate_feature(responses)
                feat_dict[obj.features[0].name] = feat_value
        else:
            print(f"More than one feature for objective: {obj.name}")
            
    feats_extra.append(feat_dict)
    eaps_extra.append(eap)

## Compute distances in parameters and features space (no extracellular)

In [ ]:
param_distances_soma = []
param_distances_apical_soma = []
param_distances_somatic_soma = []

feature_distances_soma = []

eap_distances_soma = []
for feat_dict, eap, params in zip(feats_soma, eaps_soma, params_soma):
    # params
    gt_param_values = []
    param_values = []
    gt_somatic_param_values = []
    somatic_param_values = []
    gt_apical_param_values = []
    apical_param_values = []
    
    for param_name, val in params.items():
        param_values.append(val)
        gt_param_values.append(params_release[param_name])
        if "somatic" in param_name:
            somatic_param_values.append(val)
            gt_somatic_param_values.append(params_release[param_name])
        if "apical" in param_name:
            apical_param_values.append(val)
            gt_apical_param_values.append(params_release[param_name])
    param_distances_soma.append(distance.cosine(param_values, gt_param_values))
    param_distances_apical_soma.append(distance.cosine(somatic_param_values, gt_somatic_param_values))
    param_distances_somatic_soma.append(distance.cosine(apical_param_values, gt_apical_param_values))
    
    # features
    gt_values = []
    feat_values = []
    n_in_common = 0
    for feat_name, val in feat_dict.items():
        if val is not None:
            feat_values.append(val)
            gt_values.append(features_release[feat_name])
            n_in_common += 1
    feature_distances_soma.append(distance.cosine(feat_values, gt_values) / n_in_common)
    
    # eaps
    eap_distances_soma.append(distance.cosine(eap_release.ravel(), eap.ravel()))
    
param_distances_extra = []
param_distances_apical_extra = []
param_distances_somatic_extra = []

feature_distances_extra = []

eap_distances_extra = []

for feat_dict, eap, params in zip(feats_extra, eaps_extra, params_extra):
    # params
    gt_param_values = []
    param_values = []
    gt_somatic_param_values = []
    somatic_param_values = []
    gt_apical_param_values = []
    apical_param_values = []
    
    for param_name, val in params.items():
        param_values.append(val)
        gt_param_values.append(params_release[param_name])
        if "somatic" in param_name:
            somatic_param_values.append(val)
            gt_somatic_param_values.append(params_release[param_name])
        if "apical" in param_name:
            apical_param_values.append(val)
            gt_apical_param_values.append(params_release[param_name])
    param_distances_extra.append(distance.cosine(param_values, gt_param_values))
    param_distances_apical_extra.append(distance.cosine(somatic_param_values, gt_somatic_param_values))
    param_distances_somatic_extra.append(distance.cosine(apical_param_values, gt_apical_param_values))
    
    # features
    gt_values = []
    feat_values = []
    n_in_common = 0
    for feat_name, val in feat_dict.items():
        if val is not None:
            feat_values.append(val)
            gt_values.append(features_release[feat_name])
            n_in_common += 1
    feature_distances_extra.append(distance.cosine(feat_values, gt_values) / n_in_common)
    
    # eaps
    eap_distances_extra.append(distance.cosine(eap_release.ravel(), eap.ravel()))

In [ ]:
# build dataframe
df = pd.DataFrame({"feature_set": ["soma"]*len(opt_soma) + ["extra"]*len(opt_extra)})
df['param_dist'] = param_distances_soma + param_distances_extra
df['param_dist_apical'] = param_distances_apical_soma + param_distances_apical_extra
df['param_dist_somatic'] = param_distances_somatic_soma + param_distances_somatic_extra
df['feat_dist'] = feature_distances_soma + feature_distances_extra
df['eap_dist'] = eap_distances_soma + eap_distances_extra

In [ ]:
fig1, ax1 = plt.subplots()
sns.violinplot(data=df, x='feature_set', y='param_dist', ax=ax1, alpha=0.5)
sns.stripplot(data=df, x='feature_set', y='param_dist', ax=ax1, alpha=1, palette="Greys_r")
ax1.set_title("All params")

In [ ]:
fig2, axs2 = plt.subplots(ncols=2, figsize=(9, 5))

# sns.violinplot(data=df, x='feature_set', y='param_dist_somatic', ax=axs2[0], alpha=0.5)
sns.stripplot(data=df, x='feature_set', y='param_dist_somatic', ax=axs2[0], alpha=0.5, palette="Greys_r")
# sns.violinplot(data=df, x='feature_set', y='param_dist_apical', ax=axs2[1], alpha=0.5)
sns.stripplot(data=df, x='feature_set', y='param_dist_apical', ax=axs2[1], alpha=0.5, palette="Greys_r")

axs2[0].set_title("Somatic params")
axs2[1].set_title("Apical params")
fig2.tight_layout()

In [ ]:
fig3, ax3 = plt.subplots()
# sns.violinplot(data=df, x='feature_set', y='feat_dist', ax=ax3)
sns.stripplot(data=df, x='feature_set', y='feat_dist', ax=ax3, palette="Greys_r")
ax3.set_title('All features')

In [ ]:
fig4, ax4 = plt.subplots()
sns.violinplot(data=df, x='feature_set', y='eap_dist', ax=ax4)#, alpha=0.5, ci=None)
sns.stripplot(data=df, x='feature_set', y='eap_dist', ax=ax4, palette="Greys_r")#, alpha=0.5, ci=None)
ax4.set_title("EAP")

In [ ]:
plotting.plot_multiple_eaps(responses_soma + [responses_release], eva_extra.fitness_protocols, probe,
                            protocol_name="firepattern_200", colors=["C0"] * len(responses_soma) + ["k"])
plotting.plot_multiple_eaps(responses_soma + [responses_release], eva_extra.fitness_protocols, probe,
                            protocol_name="firepattern_200", colors=["C0"] * len(responses_soma) + ["k"], 
                            norm=False)

In [ ]:
plotting.plot_multiple_eaps(responses_eap + [responses_release], eva_extra.fitness_protocols, probe,
                            protocol_name="firepattern_200", colors=["C1"] * len(responses_eap) + ["k"])
plotting.plot_multiple_eaps(responses_eap + [responses_release], eva_extra.fitness_protocols, probe,
                            protocol_name="firepattern_200", colors=["C1"] * len(responses_eap) + ["k"], 
                            norm=False)

In [ ]:
fig_soma = plotting.plot_multiple_responses(responses_list=responses_soma + [responses_release], 
                                            max_rows=5, colors=["C0"] * len(responses_soma) + ['k'], 
                                            return_fig=True)

In [ ]:
fig_extra = plotting.plot_multiple_responses(responses_list=responses_eap + [responses_release], 
                                             max_rows=5, colors=["C1"] * len(responses_eap) + ['k'], 
                                             return_fig=True)

## Plot responses

In [ ]:
# colors = {'soma': 'C1', 'extra': 'C2'}

In [ ]:
# figures_soma_intra = []
# figures_soma_extra = []
# figures_multi_intra = []
# figures_multi_extra = []
# figures_extra_intra = []
# figures_extra_extra = []
# for gt_id in range(len(gt_responses)):
#     print(f"Test model {gt_id + 1}")
#     df_fit = df[df.sample_id == str(gt_id)]
#     fitted = np.array(fitted_responses)[np.array(df_fit.index)]
    
#     color_list = []
#     feature_sets = []
#     for i in range(len(df_fit)):
#         color_list.append(colors[df_fit.iloc[i].feature_set])
#         feature_sets.append(df_fit.iloc[i].feature_set)

#     soma_idxs = np.where(np.array(feature_sets) == 'soma')
#     bap_idxs = np.where(np.array(feature_sets) == 'multiple')
#     extra_idxs = np.where(np.array(feature_sets) == 'extra')

#     fitted_soma = fitted[soma_idxs]
#     fitted_multiple = fitted[bap_idxs]
#     fitted_extra = fitted[extra_idxs]

#     color_list.append('k')
    
#     # Plot intracellular responses
#     fig_soma = plotting.plot_multiple_responses(responses_list=np.concatenate((fitted_soma, [gt_responses[gt_id]])), 
#                                                 max_rows=5, colors=[colors["soma"]] * len(fitted_soma) + ['k'], 
#                                                 return_fig=True)
#     fig_soma.suptitle(f"Test model {gt_id + 1} - 'soma' feature set\nIntracellular", fontsize=25, y=0.98)
#     fig_soma.subplots_adjust(top=0.8)
#     figures_soma_intra.append(fig_soma)
    
#     ax_eap_soma = plotting.plot_multiple_eaps(responses_list=fitted_soma, protocols=fitness_protocols,
#                                               protocol_name="Step1", probe=probe, colors="C0", norm=True)
#     ax_eap_soma = plotting.plot_eap(responses=gt_responses[gt_id], protocols=fitness_protocols,
#                                     protocol_name="Step1", probe=probe, color="k", norm=True,
#                                     ax=ax_eap_soma) 
#     ax_eap_soma.set_title(f"Test model {gt_id + 1} - 'soma' feature set\nExtracellular", fontsize=25)
#     figures_soma_extra.append(ax_eap_soma.get_figure())
    
#     fig_multiple = plotting.plot_multiple_responses(responses_list=np.concatenate((fitted_multiple, [gt_responses[gt_id]])), 
#                                                     max_rows=5, 
#                                                     colors=[colors["multiple"]] * len(fitted_multiple) + ['k'], 
#                                                     return_fig=True)
#     fig_multiple.suptitle(f"Test model {gt_id + 1} - 'multiple' feature set\nIntracellular", fontsize=25, y=0.98)
#     fig_multiple.subplots_adjust(top=0.8)
#     figures_multi_intra.append(fig_soma)
    
    
#     ax_eap_multi = plotting.plot_multiple_eaps(responses_list=fitted_multiple, protocols=fitness_protocols,
#                                                protocol_name="Step1", probe=probe, colors="C1", norm=True)
#     ax_eap_multi = plotting.plot_eap(responses=gt_responses[gt_id], protocols=fitness_protocols,
#                                      protocol_name="Step1", probe=probe, color="k", norm=True,
#                                      ax=ax_eap_multi) 
#     ax_eap_multi.set_title(f"Test model {gt_id + 1} - 'multiple' feature set\nExtracellular", fontsize=25)
#     figures_multi_extra.append(ax_eap_multi.get_figure())
    
#     fig_extra = plotting.plot_multiple_responses(responses_list=np.concatenate((fitted_extra, [gt_responses[gt_id]])), 
#                                                  max_rows=5, colors=[colors["extra"]] * len(fitted_extra) + ['k'], 
#                                                  return_fig=True)
#     fig_extra.suptitle(f"Test model {gt_id + 1} - 'extra' feature set\nIntracellular", fontsize=30, y=0.98)
#     fig_extra.subplots_adjust(top=0.8)
#     figures_extra_intra.append(fig_soma)
    
#     # Plot EAPs
#     ax_eap_extra = plotting.plot_multiple_eaps(responses_list=fitted_extra, protocols=fitness_protocols,
#                                                protocol_name="Step1", probe=probe, colors="C2", norm=True)
#     ax_eap_extra = plotting.plot_eap(responses=gt_responses[gt_id], protocols=fitness_protocols,
#                                      protocol_name="Step1", probe=probe, color="k", norm=True,
#                                      ax=ax_eap_extra) 
#     ax_eap_extra.set_title(f"Test model {gt_id + 1} - 'extra' feature set\nExtracellular", fontsize=30)
#     figures_extra_extra.append(ax_eap_extra.get_figure())
    
#     print("\n\n\n\n\n")
    
    